In [1]:
import numpy as np
np.random.seed(999999)
target_distr = np.random.rand(2)
# We now convert the random vector into a valid probability vector
target_distr /= sum(target_distr)

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

In [3]:
from qiskit import Aer, execute
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(0)
    return output_distr

def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    qc = get_var_form(params)
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    result = execute(qc, backend, shots=NUM_SHOTS).result()
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(qc))
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost

In [4]:
from qiskit.aqua.components.optimizers import COBYLA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

# Obtain the output distribution using the final parameters
qc = get_var_form(ret[0])
counts = execute(qc, backend, shots=NUM_SHOTS).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])

Target Distribution: [0.51357006 0.48642994]
Obtained Distribution: [0.5242, 0.4758]
Output Error (Manhattan Distance): 0.009459881261160819
Parameters Found: [1.52609984 1.1134972  0.67987919]


In [5]:
from qiskit.circuit.library import EfficientSU2
entanglements = ["linear", "full"]
for entanglement in entanglements:
    form = EfficientSU2(num_qubits=4, entanglement=entanglement)
    if entanglement == "linear":
        print("=============Linear Entanglement:=============")
    else:
        print("=============Full Entanglement:=============")
    # We initialize all parameters to 0 for this demonstration
    display(form.draw(fold=100))
    print()

=============Linear Entanglement:=============


┌──────────┐┌──────────┐     ┌──────────┐┌───────────┐                          ┌───────────┐»
q_0: ┤ RY(θ[0]) ├┤ RZ(θ[4]) ├──■──┤ RY(θ[8]) ├┤ RZ(θ[12]) ├───────────────────■──────┤ RY(θ[16]) ├»
     ├──────────┤├──────────┤┌─┴─┐└──────────┘└┬──────────┤┌───────────┐    ┌─┴─┐    └───────────┘»
q_1: ┤ RY(θ[1]) ├┤ RZ(θ[5]) ├┤ X ├─────■───────┤ RY(θ[9]) ├┤ RZ(θ[13]) ├────┤ X ├──────────■──────»
     ├──────────┤├──────────┤└───┘   ┌─┴─┐     └──────────┘├───────────┤┌───┴───┴───┐    ┌─┴─┐    »
q_2: ┤ RY(θ[2]) ├┤ RZ(θ[6]) ├────────┤ X ├──────────■──────┤ RY(θ[10]) ├┤ RZ(θ[14]) ├────┤ X ├────»
     ├──────────┤├──────────┤        └───┘        ┌─┴─┐    ├───────────┤├───────────┤    └───┘    »
q_3: ┤ RY(θ[3]) ├┤ RZ(θ[7]) ├─────────────────────┤ X ├────┤ RY(θ[11]) ├┤ RZ(θ[15]) ├─────────────»
     └──────────┘└──────────┘                     └───┘    └───────────┘└───────────┘             »
«     ┌───────────┐                          ┌───────────┐┌───────────┐                          
«q_0: ┤ RZ(θ[20]) ├───────────────────■──────┤ RY(θ[24]) ├┤ RZ(θ[28]) ├──────────────────────────
«     ├───────────┤┌───────────┐    ┌─┴─┐    └───────────┘├───────────┤┌───────────┐             
«q_1: ┤ RY(θ[17]) ├┤ RZ(θ[21]) ├────┤ X ├──────────■──────┤ RY(θ[25]) ├┤ RZ(θ[29]) ├─────────────
«     └───────────┘├───────────┤┌───┴───┴───┐    ┌─┴─┐    └───────────┘├───────────┤┌───────────┐
«q_2: ──────■──────┤ RY(θ[18]) ├┤ RZ(θ[22]) ├────┤ X ├──────────■──────┤ RY(θ[26]) ├┤ RZ(θ[30]) ├
«         ┌─┴─┐    ├───────────┤├───────────┤    └───┘        ┌─┴─┐    ├───────────┤├───────────┤
«q_3: ────┤ X ├────┤ RY(θ[19]) ├┤ RZ(θ[23]) ├─────────────────┤ X ├────┤ RY(θ[27]) ├┤ RZ(θ[31]) ├
«         └───┘    └───────────┘└───────────┘                 └───┘    └───────────┘└───────────┘


=============Full Entanglement:=============


┌──────────┐┌──────────┐               ┌──────────┐┌───────────┐                         »
q_0: ┤ RY(θ[0]) ├┤ RZ(θ[4]) ├──■────■────■──┤ RY(θ[8]) ├┤ RZ(θ[12]) ├─────────────────────────»
     ├──────────┤├──────────┤┌─┴─┐  │    │  └──────────┘└───────────┘┌──────────┐┌───────────┐»
q_1: ┤ RY(θ[1]) ├┤ RZ(θ[5]) ├┤ X ├──┼────┼───────■────────────■──────┤ RY(θ[9]) ├┤ RZ(θ[13]) ├»
     ├──────────┤├──────────┤└───┘┌─┴─┐  │     ┌─┴─┐          │      └──────────┘├───────────┤»
q_2: ┤ RY(θ[2]) ├┤ RZ(θ[6]) ├─────┤ X ├──┼─────┤ X ├──────────┼───────────■──────┤ RY(θ[10]) ├»
     ├──────────┤├──────────┤     └───┘┌─┴─┐   └───┘        ┌─┴─┐       ┌─┴─┐    ├───────────┤»
q_3: ┤ RY(θ[3]) ├┤ RZ(θ[7]) ├──────────┤ X ├────────────────┤ X ├───────┤ X ├────┤ RY(θ[11]) ├»
     └──────────┘└──────────┘          └───┘                └───┘       └───┘    └───────────┘»
«                            ┌───────────┐┌───────────┐                                            »
«q_0: ──────■────────■────■──┤ RY(θ[16]) ├┤ RZ(θ[20]) ├────────────────────────────────■────────■──»
«         ┌─┴─┐      │    │  └───────────┘└───────────┘┌───────────┐┌───────────┐    ┌─┴─┐      │  »
«q_1: ────┤ X ├──────┼────┼────────■────────────■──────┤ RY(θ[17]) ├┤ RZ(θ[21]) ├────┤ X ├──────┼──»
«     ┌───┴───┴───┐┌─┴─┐  │      ┌─┴─┐          │      └───────────┘├───────────┤┌───┴───┴───┐┌─┴─┐»
«q_2: ┤ RZ(θ[14]) ├┤ X ├──┼──────┤ X ├──────────┼────────────■──────┤ RY(θ[18]) ├┤ RZ(θ[22]) ├┤ X ├»
«     ├───────────┤└───┘┌─┴─┐    └───┘        ┌─┴─┐        ┌─┴─┐    ├───────────┤├───────────┤└───┘»
«q_3: ┤ RZ(θ[15]) ├─────┤ X ├─────────────────┤ X ├────────┤ X ├────┤ RY(θ[19]) ├┤ RZ(θ[23]) ├─────»
«     └───────────┘     └───┘                 └───┘        └───┘    └───────────┘└───────────┘     »
«          ┌───────────┐┌───────────┐                                       
«q_0: ──■──┤ RY(θ[24]) ├┤ RZ(θ[28]) ├───────────────────────────────────────
«       │  └───────────┘└───────────┘┌───────────┐┌───────────┐             
«q_1: ──┼────────■────────────■──────┤ RY(θ[25]) ├┤ RZ(θ[29]) ├─────────────
«       │      ┌─┴─┐          │      └───────────┘├───────────┤┌───────────┐
«q_2: ──┼──────┤ X ├──────────┼────────────■──────┤ RY(θ[26]) ├┤ RZ(θ[30]) ├
«     ┌─┴─┐    └───┘        ┌─┴─┐        ┌─┴─┐    ├───────────┤├───────────┤
«q_3: ┤ X ├─────────────────┤ X ├────────┤ X ├────┤ RY(θ[27]) ├┤ RZ(θ[31]) ├
«     └───┘                 └───┘        └───┘    └───────────┘└───────────┘

In [14]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver, NumPyMinimumEigensolver

import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit import IBMQ
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
from qiskit.chemistry.applications import MolecularGroundStateEnergy
from qiskit.chemistry.core import QubitMappingType


import os 
import time
os.environ['MPMATH_NOSAGE'] = 'true'

In [80]:
def get_qubit_op(dist):
    driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 " + str(dist) + "; H .0 .0 3.3", 
                         unit=UnitsType.ANGSTROM, charge=0, spin=1, basis='sto3g')
    molecule = driver.run()
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    
#     freeze_list = []
#     remove_list = []    
#     remove_list = [x % molecule.num_orbitals for x in remove_list]
#     freeze_list = [x % molecule.num_orbitals for x in freeze_list]
#     remove_list = [x - len(freeze_list) for x in remove_list]
#     remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
#     freeze_list += [x + molecule.num_orbitals for x in freeze_list]

    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    
#     ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
#     num_spin_orbitals -= len(freeze_list)
#     num_particles -= len(freeze_list)
#     ferOp = ferOp.fermion_mode_elimination(remove_list)
#     num_spin_orbitals -= len(remove_list)
    
    map_type = 'jordan_wigner'
    qubitOp = ferOp.mapping(map_type)
#     qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
#     shift = energy_shift + repulsion_energy
    shift = repulsion_energy
    return qubitOp, [molecule.num_alpha, molecule.num_beta], num_spin_orbitals, shift

# This is the tutorial version vs MSGE

In [ ]:
backend = BasicAer.get_backend("qasm_simulator")
dist = 0.5 
optimizer = COBYLA(maxiter=1000, tol = 0.0000001)
qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)
num_particles = 3

# VQE
initial_state = HartreeFock(
    num_spin_orbitals,
    num_particles,
    qubit_mapping='jordan_wigner',
    two_qubit_reduction = False
) 
var_form = UCCSD(
    num_orbitals=num_spin_orbitals,
    num_particles=num_particles,
    initial_state=initial_state,
    qubit_mapping='jordan_wigner',
    two_qubit_reduction = False
)
vqe = VQE(qubitOp, var_form, optimizer)

# Plain VQE
prelim = vqe.run(backend)
vqe_result = np.real(prelim['eigenvalue'] + shift)

print('Plain VQE:', vqe_result)

# # # # # # # # # # # 
# Exact results
# # # # # # # # # # # 

result = NumPyMinimumEigensolver(qubitOp).run()
print('Exact 1:', result['eigenvalue'].real + shift)
 
# Now with NumPyMinimumEigensolver
driver = PySCFDriver(atom="H .0 .0 .0;H .0 .0 0.5; H .0 .0 3.3", 
                         unit=UnitsType.ANGSTROM, charge=0, spin=1, basis='sto3g')

mgse = MolecularGroundStateEnergy(driver, NumPyMinimumEigensolver(),
                                  qubit_mapping=QubitMappingType.JORDAN_WIGNER,
                                  two_qubit_reduction=False, freeze_core=False,
                                  z2symmetry_reduction=None)
result = mgse.compute_energy()
print('MGSE:', result)

# Breaking down MSGE for VQE

In [113]:
from qiskit.chemistry.core import Hamiltonian, TransformationType, ChemistryOperator

backend = Aer.get_backend('qasm_simulator')

def cb_create_solver(num_particles, num_orbitals,
                        qubit_mapping, two_qubit_reduction, z2_symmetries):
    initial_state = HartreeFock(num_orbitals, num_particles, qubit_mapping,
                                two_qubit_reduction, z2_symmetries.sq_list)
    var_form = UCCSD(num_orbitals=num_orbitals,
                        num_particles=num_particles,
                        initial_state=initial_state,
                        qubit_mapping=qubit_mapping,
                        two_qubit_reduction=two_qubit_reduction,
                        z2_symmetries=z2_symmetries)

    vqe = VQE(var_form=var_form, optimizer=SLSQP(maxiter=500), include_custom=True)
    vqe.quantum_instance = backend
    return vqe




driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 0.5; H .0 .0 3.3", 
                         unit=UnitsType.ANGSTROM, charge=0, spin=1, basis='sto3g')
q_molecule = driver.run()


core = Hamiltonian(transformation = TransformationType.FULL,
                   qubit_mapping=QubitMappingType.JORDAN_WIGNER,
                   two_qubit_reduction=False)
operator, aux_operators = core.run(q_molecule)


# num_particles = q_molecule.num_alpha + q_molecule.num_beta
num_particles = [q_molecule.num_alpha, q_molecule.num_beta]
num_spin_orbitals = q_molecule.num_orbitals * 2

vqe = cb_create_solver(num_particles = 3,
                          num_orbitals = num_spin_orbitals,
                          qubit_mapping = 'jordan_wigner',
                          two_qubit_reduction = False,
                          z2_symmetries = core.molecule_info[ChemistryOperator.INFO_Z2SYMMETRIES])

print(vqe.compute_minimum_eigenvalue(operator, aux_operators))


{'optimal_parameters': {Parameter(θ[0]): 0.0, Parameter(θ[1]): 0.0, Parameter(θ[2]): 0.0, Parameter(θ[3]): 0.0, Parameter(θ[4]): 0.0, Parameter(θ[5]): 0.0, Parameter(θ[6]): 0.0, Parameter(θ[7]): 0.0}, 'optimal_point': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'optimal_value': -2.4452862097590176, 'optimizer_evals': 9, 'optimizer_time': 19.369983196258545, 'eigenvalue': (-2.4452862097590176+0j), 'eigenstate': {'001001': 1024}, 'aux_operator_eigenvalues': array([[2.        ],
       [0.0168457 ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.95132625]]), 'cost_function_evals': 9}


# This is to compare the operator from Hamiltonian with the qubitOp from the get_qubit_op function

In [101]:
from qiskit.chemistry.core import (Hamiltonian, TransformationType, QubitMappingType,
                                   ChemistryOperator, MolecularGroundStateResult)

backend = BasicAer.get_backend("statevector_simulator")
dist = 0.5
optimizer = SLSQP(maxiter=10)
qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)


driver = PySCFDriver(atom="H .0 .0 .0;H .0 .0 0.5; H .0 .0 3.3", 
                         unit=UnitsType.ANGSTROM, charge=0, spin=1, basis='sto3g')


core = Hamiltonian(qubit_mapping=QubitMappingType.JORDAN_WIGNER,two_qubit_reduction=False, freeze_core=False,
                                  z2symmetry_reduction=None)
operator, aux_operators = core.run(driver.run())

# # # # # # # # # # # 
# Exact results
# # # # # # # # # # # 

result = NumPyMinimumEigensolver(qubitOp).run()
print('Exact 1:', result['eigenvalue'].real + shift)

result2 = NumPyMinimumEigensolver(operator).run()
print('Exact 2:', result2['eigenvalue'].real + shift)

MGSE: -1.5216362151039462


# This is my draft for final MSGE solution, testing backends


In [133]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
backend = Aer.get_backend("qasm_simulator")
device = provider.get_backend("ibmq_16_melbourne")
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device.properties())
quantum_instance = QuantumInstance(backend=backend, 
                                   shots=10000, 
                                   noise_model=noise_model, 
                                   coupling_map=coupling_map,
                                   measurement_error_mitigation_cls=CompleteMeasFitter,
                                   cals_matrix_refresh_period=30)


backend = Aer.get_backend('qasm_simulator')
# backend = quantum_instance

optimizer = SLSQP(maxiter=500)
optimizer = SPSA(maxiter=500)

def vqe_create_solver(num_particles, num_orbitals,
                        qubit_mapping, two_qubit_reduction, z2_symmetries):
    initial_state = HartreeFock(num_orbitals, num_particles, qubit_mapping,
                                two_qubit_reduction, z2_symmetries.sq_list)
    var_form = UCCSD(num_orbitals=num_orbitals,
                        num_particles=num_particles,
                        initial_state=initial_state,
                        qubit_mapping=qubit_mapping,
                        two_qubit_reduction=two_qubit_reduction,
                        z2_symmetries=z2_symmetries)
#     var_form = EfficientSU2(6, entanglement="linear")

    vqe = VQE(var_form=var_form, optimizer=optimizer, include_custom=True)
    vqe.quantum_instance = backend
    return vqe


molecule_string = "H .0 .0 .0; H .0 .0 1.5; H .0 .0 3.3"
driver = PySCFDriver(atom=molecule_string,
                     unit=UnitsType.ANGSTROM, charge=0, spin=1, basis='sto-3g')


# # # # # # # # # # # # # # # # # # # # # #
# vqe Result
# # # # # # # # # # # # # # # # # # # # # #
msge = MolecularGroundStateEnergy(driver, qubit_mapping=QubitMappingType.JORDAN_WIGNER,
                                  two_qubit_reduction=False, freeze_core=False,
                                  z2symmetry_reduction=None)
vqe_result = msge.compute_energy(vqe_create_solver)

print('vqe result: ', vqe_result)

print(msge.solver.optimal_params)

print('\n \n')

# # # # # # # # # # # # # # # # # # # # # # #
# # Exact Result
# # # # # # # # # # # # # # # # # # # # # # #
# mgse = MolecularGroundStateEnergy(driver, NumPyMinimumEigensolver(),
#                                   qubit_mapping=QubitMappingType.JORDAN_WIGNER,
#                                   two_qubit_reduction=False, freeze_core=False,
#                                   z2symmetry_reduction=None)
# exact_result = mgse.compute_energy()
# print('exact result: ', exact_result)





ibmqfactory.load_account:WARNING:2020-10-14 10:56:36,856: Credentials are already in use. The existing account in the session will be replaced.


vqe result:  === GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -2.27101984269
  - computed part:      -2.27101984269
  - frozen energy part: 0.0
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 0.807128877262
> Total ground state energy (Hartree): -1.463890965428
  Measured:: # Particles: 3.000 S: 0.501 S^2: 0.752 M: 0.50000
 
=== DIPOLE MOMENT ===
 
* Electronic dipole moment (a.u.): [0.0  0.0  9.02199496]
  - computed part:      [0.0  0.0  9.02199496]
  - frozen energy part: [0.0  0.0  0.0]
  - particle hole part: [0.0  0.0  0.0]
~ Nuclear dipole moment (a.u.): [0.0  0.0  9.0706854]
> Dipole moment (a.u.): [0.0  0.0  0.04869044]  Total: 0.04869044
               (debye): [0.0  0.0  0.12375875]  Total: 0.12375875
[-0.89844053  1.47290342  0.15403215  1.51613595  0.90604485  1.0114973
  0.06911545 -0.65589388]

 

